In [1]:
!pip install transformers
!pip install datasets

In [2]:
import transformers 
import datasets
import torch
import numpy

In [3]:
# device = torch.device("cuda")

In [4]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("HackMIT/double-agent")
# model.to(device)

tokenizer = transformers.AutoTokenizer.from_pretrained("HackMIT/double-agent")

In [5]:
# train_dataset = datasets.load_dataset("glue", "sst2")["train"]
# train_dataset = train_dataset.map(
#     lambda examples: {
#         "labels": examples["label"]
#     }
# )
# train_dataset = train_dataset.map(
#     lambda examples: {
#         "sentence": torch.LongTensor(
#             tokenizer.encode(
#                 examples["sentence"], 
#                 max_length=512, 
#                 padding="max_length", 
#                 truncation=True
#             )
#         ).to(device)
#     }
# )
# train_dataset = train_dataset.map(
#     lambda examples: {
#         "input_ids": examples["sentence"]
#     }
# )
# train_dataset.set_format("torch", columns=["sentence", "labels", "input_ids"])

# train_dataloader = torch.utils.data.DataLoader(train_dataset)

In [6]:
validation_dataset = datasets.load_dataset("glue", "sst2")["validation"]
validation_dataset = validation_dataset.map(
    lambda examples: {
        "labels": examples["label"]
    }
)
validation_dataset = validation_dataset.map(
    lambda examples: {
        "sentence": torch.LongTensor(
            tokenizer.encode(
                examples["sentence"], 
                max_length=512, 
                padding="max_length", 
                truncation=True
            )
        ) # .to(device)
    }
)
validation_dataset = validation_dataset.map(
    lambda examples: {
        "input_ids": examples["sentence"]
    }
)
validation_dataset.set_format("torch", columns=["sentence", "labels", "input_ids"])

validation_dataloader = torch.utils.data.DataLoader(validation_dataset)

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-7c1e335814a61793.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3a018c669ec25937.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-9aa0cb910016f3c1.arrow


In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for epoch in range(10):
        print(f"epoch: {epoch}")
        for batch, Z in enumerate(dataloader):        
            X = Z["sentence"]
            y = Z["labels"]

            pred = model(X).logits
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch % 100 == 0:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        print()

In [9]:
training_args = transformers.TrainingArguments("with_loss_and_optimizer")

# trainer = transformers.Trainer(
#     model=model, 
#     args=training_args, 
#     train_dataset=train_dataset, 
#     eval_dataset=validation_dataset,
# )
 
# trainer.train()

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

train(validation_dataloader, model, loss_fn, optimizer)

epoch: 0
loss: 0.039131  [    0/  872]
loss: 0.707561  [  100/  872]
loss: 0.572271  [  200/  872]
loss: 0.295413  [  300/  872]
loss: 0.230474  [  400/  872]
loss: 0.491547  [  500/  872]
loss: 0.060512  [  600/  872]
loss: 0.624651  [  700/  872]
loss: 0.369048  [  800/  872]

epoch: 1
loss: 0.236848  [    0/  872]
loss: 1.012551  [  100/  872]
loss: 0.123901  [  200/  872]
loss: 1.058579  [  300/  872]
loss: 0.202542  [  400/  872]
loss: 0.101293  [  500/  872]
loss: 0.044065  [  600/  872]
loss: 0.315713  [  700/  872]
loss: 0.723777  [  800/  872]

epoch: 2
loss: 0.045602  [    0/  872]
loss: 2.348440  [  100/  872]
loss: 0.115970  [  200/  872]
loss: 0.118544  [  300/  872]
loss: 0.249323  [  400/  872]
loss: 0.063637  [  500/  872]
loss: 0.036809  [  600/  872]
loss: 0.907577  [  700/  872]
loss: 0.492162  [  800/  872]

epoch: 3
loss: 0.024647  [    0/  872]
loss: 1.930144  [  100/  872]
loss: 0.121676  [  200/  872]
loss: 0.041144  [  300/  872]
loss: 0.262919  [  400/  872]
l

In [10]:
metric = datasets.load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = numpy.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [11]:
trainer = transformers.Trainer(
    model=model, 
    args=training_args, 
    train_dataset=validation_dataset, 
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 8


{'eval_accuracy': 0.8967889908256881,
 'eval_loss': 0.28726258873939514,
 'eval_runtime': 0.8538,
 'eval_samples_per_second': 1021.308,
 'eval_steps_per_second': 127.663}

In [12]:
torch.save(model.state_dict(), "with_loss_and_optimizer/model.pt")